In [1]:
import json
import os
import sys
import math
import datetime
import pathlib
import numpy as np

dir_path = pathlib.Path(os.path.abspath('')).resolve()
print(dir_path)
HOME_DIR = str(dir_path).split('/OpenFormat-Public')[0]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

PROJ_SRC_DIR = f'{HOME_DIR}/OpenFormat-Public'
sys.path.insert(1, f'{PROJ_SRC_DIR}')
from python.scripts.utils import *
num_rows = 1000 * 1000
num_cols = 20
workload_list = ['core', 'geo', 'classic', 'log', 'ml', 'bi']

/home/haoyuq/EvaluationOfColumnarFormats/OpenFormat/OpenFormat-Public/python/general


In [2]:
HOME_DIR = str(dir_path).split('/OpenFormat')[0]
scan_exec_pq = f'{HOME_DIR}/arrow-private/arrow-Public/cpp/out/build/openformat-release-static/release/parquet-scan-columnbatch'

# Predefined workloads, size and scan time

In [3]:
%%time
# generate csv files
os.chdir(f'{PROJ_SRC_DIR}/benchmark/generator_v2')
!pwd
for wl in workload_list:
    base_name = f'{wl}_r{num_rows}_c{num_cols}'
    os.system(f'python3 gen_workloads.py \
        {wl} {num_rows} {num_cols} {base_name}')
    os.system(f'cp -r {base_name} {dir_path} && \
        rm -r {base_name}')
os.chdir(dir_path)
!pwd

/home/haoyuq/EvaluationOfColumnarFormats/OpenFormat/OpenFormat-Public/benchmark/generator_v2
Finish core dataset! 🎉🎉🎉🎉🎉🎉
Generating Done!!
Finish workload: geo! 🎉🎉🎉🎉🎉🎉
Generating Done!!
Finish workload: classic! 🎉🎉🎉🎉🎉🎉
Generating Done!!
🤔 INFO: too many collisions in int_unique_list, 19734 values just use (max_value + 1)!
🤔 INFO: too many collisions in int_unique_list, 342213 values just use (max_value + 1)!
Finish workload: log! 🎉🎉🎉🎉🎉🎉
Generating Done!!
Finish workload: ml! 🎉🎉🎉🎉🎉🎉
Generating Done!!
🤔 INFO: too many collisions in int_unique_list, 7 values just use (max_value + 1)!
Finish workload: bi! 🎉🎉🎉🎉🎉🎉
Generating Done!!
/home/haoyuq/EvaluationOfColumnarFormats/OpenFormat/OpenFormat-Public/python/general
CPU times: user 28.9 ms, sys: 19.1 ms, total: 48 ms
Wall time: 12min 19s


In [4]:
pq_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/pq_default.json')
orc_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/orc_default.json')
pq_name_suffix = '_' + "_".join([str(i) for i in list(pq_config[0].values())]) + '.parquet'
orc_name_suffix = '_' + "_".join([str(i) for i in list(orc_config[0].values())]) + '.orc'
print(pq_config)

loading config from /home/haoyuq/EvaluationOfColumnarFormats/OpenFormat/OpenFormat-Public/python/experiments/pq_default.json ...
loading config from /home/haoyuq/EvaluationOfColumnarFormats/OpenFormat/OpenFormat-Public/python/experiments/orc_default.json ...
[{'compression': 'SNAPPY', 'use_dictionary': True, 'dictionary_pagesize_limit': 1048576, 'row_group_size': 1048576, 'version': '2.6', 'data_page_version': '2.0'}]


## BtrBlocks files

In [5]:
# create btrblock schema file
import yaml
os.chdir(f'{PROJ_SRC_DIR}/python')
for wl in workload_list:
    base_name = f'{wl}_r{num_rows}_c{num_cols}'
    arrow_schema_name = f'general/{base_name}/gen_data/{base_name}_arrow_schema.txt'
    with open(arrow_schema_name, 'r') as f:
        lines = f.readlines()
    columns = []
    cnt = 0
    for line in lines:
        if 'message' in line or '}' in line or len(line)==0:
            print(line)
            continue
        col_name = f'col{cnt}'
        col_type = None
        if 'String' in line:
            col_type = 'string'
        elif 'int' in line:
            col_type = 'integer'
        elif 'double' in line:
            col_type = 'double'
        if col_type is None:
            print(line)
            col_type = 'skip'
        cnt += 1
        columns.append({'name': col_name, 'type':col_type})
    with open(f'general/{base_name}/gen_data/{base_name}_btrblocks_schema.yaml','w') as f:
        yaml.dump({'columns':columns}, f)

message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}
message schema {

}


In [6]:
os.chdir(f'{PROJ_SRC_DIR}/python')
for wl in workload_list:
    base_name = f'{wl}_r{num_rows}_c{num_cols}'
    print(base_name)
    # os.system(f'cp general/{base_name}/gen_data/{base_name}.csv data_gen')
    # os.system(f'parquet-fromcsv -s general/{base_name}/gen_data/{base_name}_arrow_schema.txt -i \
    # general/{base_name}/gen_data/{base_name}.csv -o general/{base_name}.parquet')
    # os.system(f'python3 scripts/parquet_exp.py {base_name} pq_default -r=false -o=false')
    orc_schema = open(f'general/{base_name}/gen_data/{base_name}_orc_schema.txt', 'r').read()
    os.system(f'{HOME_DIR}/orc/orc/build/tools/src/csv-import \"{orc_schema}\" \
            general/{base_name}/gen_data/{base_name}.csv {base_name}.orc')
    # os.system(f'mv {base_name}{pq_name_suffix} general/{base_name}.parquet')
    os.system(f'mv {base_name}.orc general/{base_name}.orc')
os.chdir(f'{PROJ_SRC_DIR}/python/general')

core_r1000000_c20
[2024-03-06 03:13:33] Start importing Orc file...
[2024-03-06 03:13:40] Finish importing Orc file.
[2024-03-06 03:13:40] Total writer elasped time: 1.66484s.
[2024-03-06 03:13:40] Total writer CPU time: 1.66364s.
geo_r1000000_c20
[2024-03-06 03:13:41] Start importing Orc file...
[2024-03-06 03:13:51] Finish importing Orc file.
[2024-03-06 03:13:51] Total writer elasped time: 3.14967s.
[2024-03-06 03:13:51] Total writer CPU time: 3.14743s.
classic_r1000000_c20
[2024-03-06 03:13:52] Start importing Orc file...
[2024-03-06 03:14:03] Finish importing Orc file.
[2024-03-06 03:14:03] Total writer elasped time: 3.10736s.
[2024-03-06 03:14:03] Total writer CPU time: 3.09929s.
log_r1000000_c20
[2024-03-06 03:14:03] Start importing Orc file...
[2024-03-06 03:14:10] Finish importing Orc file.
[2024-03-06 03:14:10] Total writer elasped time: 1.21241s.
[2024-03-06 03:14:10] Total writer CPU time: 1.21138s.
ml_r1000000_c20
[2024-03-06 03:14:10] Start importing Orc file...
[2024-03-

In [8]:
# generating files for btrblocks
os.chdir(f'{PROJ_SRC_DIR}/python')
for wl in workload_list:
    base_name = f'{wl}_r{num_rows}_c{num_cols}'
    os.system(f'mkdir general/{base_name}/gen_data/binary')
    os.system(f'mkdir general/{base_name}/gen_data/btr')
    
    os.system(f'~/btrblocks/build/csvtobtr -create_binary -binary general/{base_name}/gen_data/binary/ -csv general/{base_name}/gen_data/{base_name}.csv -yaml general/{base_name}/gen_data/{base_name}_btrblocks_schema.yaml')
    os.system(f'~/btrblocks/build/csvtobtr -create_btr -btr general/{base_name}/gen_data/btr/ -binary general/{base_name}/gen_data/binary/ -csv general/{base_name}/gen_data/{base_name}.csv -yaml general/{base_name}/gen_data/{base_name}_btrblocks_schema.yaml')
os.chdir(f'{PROJ_SRC_DIR}/python/general')

mkdir: cannot create directory ‘general/core_r1000000_c20/gen_data/binary’: File exists
mkdir: cannot create directory ‘general/core_r1000000_c20/gen_data/btr’: File exists


[2024-03-06 03:43:39.155] [info] Creating binary files in general/core_r1000000_c20/gen_data/binary/
column name = col0 , type = integer
column name = col1 , type = integer
column name = col2 , type = integer
column name = col3 , type = integer
column name = col4 , type = integer
column name = col5 , type = integer
column name = col6 , type = integer
column name = col7 , type = integer
column name = col8 , type = double
column name = col9 , type = double
column name = col10 , type = double
column name = col11 , type = double
column name = col12 , type = string
column name = col13 , type = string
column name = col14 , type = string
column name = col15 , type = string
column name = col16 , type = string
column name = col17 , type = string
column name = col18 , type = string
column name = col19 , type = string
[haoyu.debug.Parser] writing binary filegeneral/core_r1000000_c20/gen_data/binary/1_col0.bitmap
Writing binary file : general/core_r1000000_c20/gen_data/binary/1_col0.bitmap
[haoyu.

mkdir: cannot create directory ‘general/geo_r1000000_c20/gen_data/binary’: File exists
mkdir: cannot create directory ‘general/geo_r1000000_c20/gen_data/btr’: File exists


[2024-03-06 03:43:55.301] [info] Creating binary files in general/geo_r1000000_c20/gen_data/binary/
column name = col0 , type = integer
column name = col1 , type = integer
column name = col2 , type = integer
column name = col3 , type = integer
column name = col4 , type = integer
column name = col5 , type = integer
column name = col6 , type = integer
column name = col7 , type = double
column name = col8 , type = string
column name = col9 , type = string
column name = col10 , type = string
column name = col11 , type = string
column name = col12 , type = string
column name = col13 , type = string
column name = col14 , type = string
column name = col15 , type = string
column name = col16 , type = string
column name = col17 , type = string
column name = col18 , type = string
column name = col19 , type = string
[haoyu.debug.Parser] writing binary filegeneral/geo_r1000000_c20/gen_data/binary/1_col0.bitmap
Writing binary file : general/geo_r1000000_c20/gen_data/binary/1_col0.bitmap
[haoyu.debu

mkdir: cannot create directory ‘general/classic_r1000000_c20/gen_data/binary’: File exists
mkdir: cannot create directory ‘general/classic_r1000000_c20/gen_data/btr’: File exists


...

mkdir: cannot create directory ‘general/ml_r1000000_c20/gen_data/binary’: File exists
mkdir: cannot create directory ‘general/ml_r1000000_c20/gen_data/btr’: File exists


[2024-03-06 03:45:04.536] [info] Creating binary files in general/ml_r1000000_c20/gen_data/binary/
column name = col0 , type = integer
column name = col1 , type = integer
column name = col2 , type = integer
column name = col3 , type = integer
column name = col4 , type = integer
column name = col5 , type = double
column name = col6 , type = double
column name = col7 , type = double
column name = col8 , type = double
column name = col9 , type = double
column name = col10 , type = double
column name = col11 , type = double
column name = col12 , type = double
column name = col13 , type = double
column name = col14 , type = string
column name = col15 , type = string
column name = col16 , type = string
column name = col17 , type = string
column name = col18 , type = string
column name = col19 , type = string
[haoyu.debug.Parser] writing binary filegeneral/ml_r1000000_c20/gen_data/binary/1_col0.bitmap
Writing binary file : general/ml_r1000000_c20/gen_data/binary/1_col0.bitmap
[haoyu.debug.wri

mkdir: cannot create directory ‘general/bi_r1000000_c20/gen_data/binary’: File exists
mkdir: cannot create directory ‘general/bi_r1000000_c20/gen_data/btr’: File exists


[2024-03-06 03:45:25.777] [info] Creating binary files in general/bi_r1000000_c20/gen_data/binary/
column name = col0 , type = integer
column name = col1 , type = integer
column name = col2 , type = integer
column name = col3 , type = integer
column name = col4 , type = integer
column name = col5 , type = integer
column name = col6 , type = integer
column name = col7 , type = integer
column name = col8 , type = integer
column name = col9 , type = integer
column name = col10 , type = double
column name = col11 , type = double
column name = col12 , type = double
column name = col13 , type = double
column name = col14 , type = string
column name = col15 , type = string
column name = col16 , type = string
column name = col17 , type = string
column name = col18 , type = string
column name = col19 , type = string
[haoyu.debug.Parser] writing binary filegeneral/bi_r1000000_c20/gen_data/binary/1_col0.bitmap
Writing binary file : general/bi_r1000000_c20/gen_data/binary/1_col0.bitmap
[haoyu.debu

!!! compressed is larger than raw: 
for : values - scheme = DICT difference = 13. Falling back to uncompressed.
!!! compressed is larger than raw: 
for : ? - scheme = PSEUDODECIMAL difference = 1000. Falling back to uncompressed.
!!! compressed is larger than raw: 
for : ? - scheme = PSEUDODECIMAL difference = 1000. Falling back to uncompressed.
!!! compressed is larger than raw: 
for : ? - scheme = PSEUDODECIMAL difference = 1000. Falling back to uncompressed.


In [9]:
# running experiments
os.chdir(f'{PROJ_SRC_DIR}/python/general')
os.system('rm outputs/stats.json')
for i in range(10):
    for wl in workload_list:
        base_name = f'{wl}_r{num_rows}_c{num_cols}'
        os.system('sudo \sync; sudo sh -c "/usr/bin/echo 3 > /proc/sys/vm/drop_caches"')
        time.sleep(1)
        pq_read = float(os.popen(f'''{scan_exec_pq} \
                --batch_size=1024 ./{base_name}.parquet''').read().split('\n')[0].split(' ')[-2])
        orc_read = float(os.popen(f'''{HOME_DIR}/orc/orc/build/tools/src/orc-scan \
                -b 1024 ./{base_name}.orc''').read().split('\n')[0].split(' ')[-1])
        btrblocks_out = os.popen(f'''~/btrblocks/build/decompression-speed -output_summary \
                    -btr ./{base_name}/gen_data/btr/ -threads 1''').read().split('\n')[-2],
        btrblocks_read = float(btrblocks_out.split(' ')[-4])
        btrblocks_size = float(btrblocks_out.split(' ')[1]),
        btrblocks_size_with_metadata = btrblocks_size + os.path.getsize(f'./{base_name}/gen_data/btr/metadata')
        orc_size = os.path.getsize(f'{base_name}.orc')
        pq_size = os.path.getsize(f'{base_name}.parquet')
        output_stats = {}
        output_stats['workload'] = wl
        output_stats['i'] = i
        output_stats['file'] = 'parquet'
        output_stats['size'] = pq_size
        output_stats['read_time'] = pq_read
        parse_output(output_stats)
        output_stats['file'] = 'orc'
        output_stats['size'] = orc_size
        output_stats['read_time'] = orc_read
        output_stats['file'] = 'btrblocks'
        output_stats['size'] = btrblocks_size
        output_stats['read_time'] = btrblocks_read
        output_stats['size_with_metadata'] = btrblocks_size_with_metadata
        parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format('general_exp', timestamp))

sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper
sudo: a password is required
sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper
sudo: a password is required
Parquet error: IOError: Failed to open local file './core_r1000000_c20.parquet'. Detail: [errno 2] No such file or directory


IndexError: list index out of range

# Predefined workloads, filter (select)

In [ ]:
# generate filters and store in dir
os.chdir(dir_path)
for wl in workload_list:
    base_name = f'{wl}_r{num_rows}_c{num_cols}'
    filter_dir = f'{wl}_filters'
    os.makedirs(filter_dir, exist_ok=True)
    os.chdir(filter_dir)
    os.system(f'rm -rf point*.csv range*.csv')
    config_name = f'{PROJ_SRC_DIR}/benchmark/generator_v2/filter_config/{wl}_filter.yaml'
    os.system(f'''python3 {PROJ_SRC_DIR}/benchmark/generator_v2/filter_generator.py \
    ../{base_name}/gen_data/{base_name}.csv ../{base_name}/configs/table_config.json \
        {config_name}''')
    os.chdir(dir_path)

In [ ]:
# running experiments
import yaml
import itertools
os.system('rm outputs/stats.json')
output_stats = {}
for wl in workload_list:
    print(f'begin wl: {wl}')
    output_stats['wl'] = wl
    base_name = f'{wl}_r{num_rows}_c{num_cols}'
    filter_dir = f'{wl}_filters'
    config_name = f'{PROJ_SRC_DIR}/benchmark/generator_v2/filter_config/{wl}_filter.yaml'
    config = yaml.safe_load(open(config_name))
    config_list = config['filter_config']
    proj_type = 'one'
    for (i, fmt, config) in itertools.product(range(5), ['parquet', 'orc'], config_list):
        output_stats['format'] = fmt
        output_stats['i'] = i
        output_stats['proj_type'] = proj_type
        expectation = config.get('selectivity_expectation')
        output_stats['selectivity'] = expectation
        range_exist = True
        point_exist = True
        try:
            range_filters = pd.read_csv(f'{filter_dir}/range_{expectation}.csv')
        except:
            range_exist = False
        try:
            point_filters = pd.read_csv(f'{filter_dir}/point_{expectation}.csv')
        except:
            point_exist = False
        if range_exist:
            output_stats['filter_type'] = 'range'
            for index, row in range_filters.iterrows():
                val1 = np.int64(float(row['val1'])) if row['dtype'] == 'int' else row['val1']
                val2 = np.int64(float(row['val2'])) if row['dtype'] == 'int' else row['val2']
                os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
                if fmt == 'parquet':
                    cmd = f"parquet-filterscan-noeval -f \
                        {base_name}.parquet -i {row['col_idx']} -l {val1} -r \
                            {val2} -e true"
                    output_stats['time'] = os.popen(cmd).read().split('\n')[0].split(' ')[-2]
                    output_stats['time_preload'] = os.popen(cmd).read().split('\n')[0].split(' ')[-2]
                else:
                    cmd = f"{HOME_DIR}/orc/build/c++/test/FilterExp \
                    {base_name}.orc f{row['col_idx']} {row['dtype']} range \
                        {val1} {val2} {proj_type} e"
                    output_stats['time'] = float(os.popen(cmd).read().split('\n')[0].split(' ')[-1])*1000
                    output_stats['time_preload'] = float(os.popen(cmd).read().split('\n')[0].split(' ')[-1])*1000
                parse_output(output_stats)
        if point_exist:
            output_stats['filter_type'] = 'point'
            for index, row in point_filters.iterrows():
                val1 = np.int64(float(row['value'])) if row['dtype'] == 'int' else row['value']
                val2 = np.int64(float(row['value'])) if row['dtype'] == 'int' else row['value']
                os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
                if fmt == 'parquet':
                    cmd = f"parquet-filterscan-noeval -f \
                        {base_name}.parquet -i {row['col_idx']} -l {val1} -r \
                            {val2} -e true"
                    output_stats['time'] = os.popen(cmd).read().split('\n')[0].split(' ')[-2]
                    output_stats['time_preload'] = os.popen(cmd).read().split('\n')[0].split(' ')[-2]
                else:
                    cmd = f"{HOME_DIR}/orc/build/c++/test/FilterExp \
                    {base_name}.orc f{row['col_idx']} {row['dtype']} range \
                        {val1} {val2} {proj_type} e"
                    output_stats['time'] = float(os.popen(cmd).read().split('\n')[0].split(' ')[-1])*1000
                    output_stats['time_preload'] = float(os.popen(cmd).read().split('\n')[0].split(' ')[-1])*1000
                parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format(f'general_filter', timestamp))